# Test how to make predictions using saved tfmodel


In [1]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib
from joblib import dump, load

In [2]:
encoder = joblib.load('../Resources/encoder.joblib')
scaler = joblib.load('../Resources/scaler.joblib')

In [3]:
file = ("../Resources/tensorflowmodel.keras")
model = tf.keras.models.load_model(file)

In [5]:
import pandas as pd

In [43]:
# Define the new data
new_data = {'person_age': 21, 'person_income': 9600, 'person_home_ownership': 'MORTGAGE', 'person_emp_length': 1, 'loan_intent': 'MEDICAL', 'loan_grade': 'C', 'loan_amnt': 9600, 'loan_int_rate': 12.87, 'loan_percent_income': 1, 'cb_person_default_on_file': 'N', 'cb_person_cred_hist_length': 1}

# Convert to DataFrame
new_df = pd.DataFrame([new_data])

# Convert categorical variables to one-hot encoding
categorical_columns = ['person_home_ownership', 'loan_intent', 'loan_grade', 'cb_person_default_on_file']
new_df = pd.get_dummies(new_df, columns=categorical_columns)

# Ensure all columns present in the training data are in the new data
training_categorical_columns = ['person_home_ownership_MORTGAGE', 'person_home_ownership_OTHER', 
                                 'person_home_ownership_OWN', 'person_home_ownership_RENT', 
                                 'loan_intent_DEBTCONSOLIDATION', 'loan_intent_EDUCATION', 
                                 'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL', 
                                 'loan_intent_PERSONAL', 'loan_intent_VENTURE', 'loan_grade_A', 
                                 'loan_grade_B', 'loan_grade_C', 'loan_grade_D', 'loan_grade_E', 
                                 'loan_grade_F', 'loan_grade_G', 'cb_person_default_on_file_N', 
                                 'cb_person_default_on_file_Y']

missing_cols = set(training_categorical_columns).difference(new_df.columns)
for col in missing_cols:
    new_df[col] = 0

# Reorder the columns to match the order in the training data
new_df = new_df[['person_age', 'person_income', 'person_emp_length', 
                                                'loan_amnt', 'loan_int_rate','loan_percent_income', 'cb_person_cred_hist_length'] + training_categorical_columns  ]

pd.set_option('display.max_columns', None)

# Now new_df contains the preprocessed data ready for prediction
new_df

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,cb_person_default_on_file_N,cb_person_default_on_file_Y
0,21,9600,1,9600,12.87,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0


In [51]:
scaled_data = scaler.fit_transform(new_df)

In [48]:
predictions = model.predict(scaled_data)

1/1 [==============================] - 0s 25ms/step


In [52]:
# Assuming it's a binary classification, you might want to threshold probabilities to get binary predictions
binary_predictions = (predictions > 0.5).astype(int)

# Print the predictions
print("Predictions:", binary_predictions[0][0])

Predictions: 0
